In [2]:
import numpy as np
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from tqdm import tqdm

In [3]:
class Models(nn.Module):
    def __init__(self, num1,num2, num_models, activation_fn=nn.Sigmoid()):
        super(Models, self).__init__()
        self.models = nn.ModuleDict({
            f"fc{i+1}": nn.Sequential(
                nn.Linear(num1, num2),
                activation_fn,
                nn.Linear(num2,1),
                activation_fn
            ) for i in range(num_models)
        })

    def forward(self, x):
        outputs =  [model(x) for _, model in self.models.items()]
        data = torch.cat(outputs, dim=-1)
        return data

def train(model, optimizer, x_train, y_train,max_epochs=1000,lam=1):
    x_train = Variable(x_train, requires_grad=False)
    y_train = Variable(y_train, requires_grad=False)


    for epoch in tqdm(range(max_epochs)):
        optimizer.zero_grad()
        outputs= model(x_train)
        # loss1 = nn.CrossEntropyLoss()
        loss2 = 0
        # if lam!=0:
        #     for i in range(y_train.shape[1]):
        #         for k in range(i+1,y_train.shape[1]):
        #             loss2 += abs(model.models[f'fc{i+1}'][0].weight-model.models[f'fc{k+1}'][0].weight).sum()
        #             loss2 += abs(model.models[f'fc{i+1}'][2].weight-model.models[f'fc{k+1}'][2].weight).sum()
        if lam != 0:
            weights = [model.models[f'fc{i+1}'][0].weight for i in range(y_train.shape[1])]
            # 对所有权重层两两之间计算差异的绝对值和
            for i in range(len(weights)):
                for j in range(i + 1, len(weights)):
                    loss2 += torch.abs(weights[i] - weights[j]).sum()

        loss = ((outputs-y_train)**2).sum()+lam*loss2
        
        loss.backward()
        optimizer.step()

        if (epoch+1)%100 == 0:
            print(f'Epoch {epoch+1}, Loss: {(loss).item()}')

def px(x):
    p = x@torch.inverse(x.T@x)@x.T
    return torch.eye(p.shape[0],p.shape[1])-p

def rmse(matrix1, matrix2):
    # 确保两个矩阵具有相同的维度
    if matrix1.shape != matrix2.shape:
        raise ValueError("Both matrices must have the same dimensions.")
    # 计算差异
    diff = matrix1 - matrix2
    # 计算均方误差（MSE）
    mse = torch.mean(diff ** 2)
    # 计算RMSE
    rmse = torch.sqrt(mse)
    return rmse.item()

def step2(x,theta,y,M,lam1=1,lam2=1,a=1):
    beta = torch.inverse(x.T@x+lam1*torch.eye(x.shape[1],x.shape[1]))@x.T@(torch.mul(torch.mul(M,theta),y))
    b1 = 1/(1+2*(y.shape[0]*y.shape[1]*lam2/2)*(1-a))
    b2 = px(x)@(torch.mul(torch.mul(M,theta),y))
    a1,b,c = torch.linalg.svd(b2, full_matrices=False)
    b = b-a*(y.shape[0]*y.shape[1])*lam2/2
    for i in range(b.shape[0]):
        b[i] = b[i] if b[i].item()>0 else 0
    b = torch.diag_embed(b, 0, -2, -1)[:a1.size(0), :c.size(1)]
    b = a1@b@c*b1
    return beta,b



## Simulation

In [4]:
m = 100 # 样本量
n1 = 4 # 协变量矩阵特征纬度
n2 = 10 # 观测矩阵纬度



w = torch.normal(0.3,1,(n1,n2))
b = torch.normal(-1.5,0.01,(n2,1)).reshape(-1)
for i in range(1,n2):
    w[:,i] = w[:,0]+torch.normal(0,0.2,(n1,1)).reshape(-1)
x = torch.normal(0,1,(m,n1))#生成固定分布的协变量矩阵
theta = torch.sigmoid(x@w+b)

w1 = torch.normal(0.3,0.01,(n1,n2))
b1 = px(x)@torch.randn(m,10)@torch.randn(10,n2)
y = x@w1+b1
noise = torch.normal(0,((y-torch.mean(y))**2).sum()/(m*n2-1),(m,n2))

#定义示性矩阵
M = torch.zeros((m,n2))
for i in range(m):
    for j in range(n2):
        M[i,j] = 1 if np.random.uniform(0,1) <= theta[i,j] else 0



## Step1

In [11]:
model = Models(n1,n1+2,n2)
opt = torch.optim.Adam(model.parameters(),lr=0.01)
train(model,opt,x,M,lam=0,max_epochs=1000)
mae = abs(model(x)-theta).sum()/(m*n2)
print(f"平均误差:{mae}")

 18%|█▊        | 184/1000 [00:00<00:01, 616.92it/s]

Epoch 100, Loss: 147.68936157226562
Epoch 200, Loss: 135.5389404296875


 38%|███▊      | 380/1000 [00:00<00:00, 643.02it/s]

Epoch 300, Loss: 128.22332763671875
Epoch 400, Loss: 121.49529266357422


 58%|█████▊    | 576/1000 [00:00<00:00, 644.68it/s]

Epoch 500, Loss: 115.40435028076172
Epoch 600, Loss: 110.0126724243164


 77%|███████▋  | 773/1000 [00:01<00:00, 651.04it/s]

Epoch 700, Loss: 104.27096557617188
Epoch 800, Loss: 97.87535858154297


100%|██████████| 1000/1000 [00:01<00:00, 636.40it/s]

Epoch 900, Loss: 92.19554901123047
Epoch 1000, Loss: 87.27667999267578
平均误差:0.15355032682418823


## Step2

### 计算缺失位置误差

In [15]:
beta,b = step2(x,model(x)**(-1),torch.mul(y+noise,M),M,lam1=1000000,lam2=0.8,a=0.8)
rmse(torch.mul(x@beta+b,M),torch.mul(y,M))

10.033164978027344

### 计算全部位置误差

In [16]:
rmse(x@beta+b,y)

10.422530174255371